In [1]:
import sys

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window

In [2]:
# ===== Шаг 1: инициализация SparkSession =====
spark = (
    SparkSession.builder
    .appName("TopLanguagesByYear")
    .master("local[1]")  # один исполнитель, чтобы не плодить Python-воркеры
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.pyspark.python", sys.executable)
    .config("spark.pyspark.driver.python", sys.executable)
    .config("spark.python.worker.timeout", "300000")
    .getOrCreate()
)
sc = spark.sparkContext
sc.setLogLevel("WARN")

In [3]:
# ===== Шаг 2: загрузка списка языков и broadcast =====
languages_csv_path = r"../data/programming-languages.csv"
languages_df = spark.read.csv(languages_csv_path, header=True)
lang_col = languages_df.columns[0]
languages_set = {row[lang_col].lower() for row in languages_df.collect()}
languages_bc = sc.broadcast(languages_set)

print(f"Загружено {len(languages_set)} языков программирования.")

Загружено 698 языков программирования.


In [86]:
# ===== Шаг 3: чтение XML как текста, фильтрация строк с тегами =====
posts_txt = spark.read.text(r"../data/posts_sample.xml")
filtered = posts_txt.filter(
    F.col("value").contains("CreationDate") & F.col("value").contains("Tags=")
)

print(f"Отфильтровано {filtered.count()} строк с тегами и датами.")

Отфильтровано 18057 строк с тегами и датами.


In [87]:
# ===== Шаг 4: извлечение года и сырой строки тегов =====
raw = filtered.select(
    F.regexp_extract("value", r'CreationDate="(\d{4})', 1).cast("int").alias("year"),
    F.regexp_extract("value", r'Tags="([^"]*)"', 1).alias("raw_tags")
).filter(
    (F.col("year") >= 2010) & (F.col("year") <= 2020)  # Фильтрация по годам
)

print(f"Извлечено {raw.count()} записей с годом и тегами, отфильтрованных по годам.")

Извлечено 17644 записей с годом и тегами, отфильтрованных по годам.


In [88]:
# ===== Шаг 5: преобразование raw_tags в массив тегов =====
tags_df = raw.withColumn(
    "tag_str",
    F.regexp_replace("raw_tags", "&lt;|&gt;", ",")
).withColumn(
    "tags",
    F.split(F.expr("trim(both ',' from tag_str)"), ",")
).select("year", "tags")

print(f"Преобразовано в массив тегов для {tags_df.count()} записей.")

Преобразовано в массив тегов для 17644 записей.


In [89]:
# ===== Шаг 6: explode + фильтрация по списку языков =====
exploded = tags_df.select(
    "year",
    F.explode("tags").alias("language")
).withColumn(
    "language", F.lower("language")
).filter(
    F.col("language").isin(languages_bc.value)
)

print(f"После применения explode и фильтрации по языкам, получено {exploded.count()} строк.")

После применения explode и фильтрации по языкам, получено 8054 строк.


In [90]:
# ===== Шаг 7: группировка и подсчёт упоминаний =====
counts = exploded.groupBy("year", "language").count()

print(f"Подсчитано упоминаний для {counts.count()} языков по годам.")

Подсчитано упоминаний для 438 языков по годам.


In [91]:
# ===== Шаг 8: присвоение ранга в годовом окне =====
window_spec = Window.partitionBy("year").orderBy(F.desc("count"))
ranked = counts.withColumn("rank", F.row_number().over(window_spec)).filter(F.col("rank") <= 10)

print(f"Присвоены ранги, выбраны топ-10 языков для каждого года.")

Присвоены ранги, выбраны топ-10 языков для каждого года.


In [92]:
# ===== Шаг 9: pivot по рангу и сбор в массив =====
pivoted = ranked.groupBy("year").pivot("rank", list(range(1, 11))).agg(F.first("language"))
final_df = pivoted.select(
    "year",
    F.array(*[F.col(str(i)) for i in range(1, 11)]).alias("top_languages")
).orderBy("year")

print(f"Сформирован итоговый DataFrame с топ-10 языками по годам.")

Сформирован итоговый DataFrame с топ-10 языками по годам.


In [93]:
# ===== Шаг 10: вывод результатов =====
final_df.show(truncate=False)

+----+--------------------------------------------------------------------------------+
|year|top_languages                                                                   |
+----+--------------------------------------------------------------------------------+
|2010|[java, php, javascript, python, objective-c, c, ruby, delphi, applescript, r]   |
|2011|[php, java, javascript, python, objective-c, c, ruby, perl, delphi, bash]       |
|2012|[php, javascript, java, python, objective-c, ruby, c, bash, r, scala]           |
|2013|[php, javascript, java, python, objective-c, c, ruby, r, bash, scala]           |
|2014|[javascript, java, php, python, objective-c, c, r, ruby, bash, matlab]          |
|2015|[javascript, java, php, python, r, c, objective-c, ruby, matlab, scala]         |
|2016|[javascript, java, php, python, r, c, ruby, bash, scala, matlab]                |
|2017|[javascript, java, python, php, r, c, typescript, objective-c, ruby, powershell]|
|2018|[python, javascript, java,

In [94]:
# ===== Шаг 11: сохранение в Parquet =====
output_path = r"top_10_languages_by_year.parquet"
final_df.write.mode("overwrite").parquet(output_path)

print(f"Результаты сохранены в Parquet в {output_path}.")

Результаты сохранены в Parquet в top_10_languages_by_year.parquet.


In [95]:
spark.stop()